In [ ]:
import numpy as np
import pandas as pd
import os
import torch 
from torch import nn, optim 
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torch.utils.data
import matplotlib.pyplot as plt
import time
import itertools
import glob 
from PIL import Image
import csv 
import cv2
import re
!pip install timm
!pip install effdet
import glob
import shutil
import os
import json
import yaml

Link to COCO format tutorial

https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch


In [ ]:
import zipfile

!git clone https://github.com/SarthakJaingit/Fruits-and-Spot-Detection.git
%cd "Fruits-and-Spot-Detection"
with zipfile.ZipFile("/content/Fruits-and-Spot-Detection/LatestFruit Defects Dataset .zip", 'r') as zip_ref:
     zip_ref.extractall('/content/')

Cloning into 'Fruits-and-Spot-Detection'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 267 (delta 34), reused 0 (delta 0), pack-reused 196
Receiving objects: 100% (267/267), 34.02 MiB | 29.40 MiB/s, done.
Resolving deltas: 100% (127/127), done.
/content/Fruits-and-Spot-Detection


In [ ]:
#For one strawberry batch please drop watermark rows
strawberry_csv_batch_3 = pd.read_csv("/content/Fruit Defects Dataset /Train/FreshStrawberries/Fresh StrawBerry Batch 3 Labeled/FreshStrawberryBatch3Labels.csv", header = None)
strawberry_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/FreshStrawberries/Fresh StrawBerry Batch 2 Labeled/FreshStrawberriesBatch2Labels.csv", header = None)
strawberry_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/FreshStrawberries/Fresh StrawBerry Batch 1 Labeled/Strawberrybatch1.csv", header = None)
rottenApple_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/RottenApples/RottenAppleBatch1Labeled/RottenAppleBatch1Labels.csv", header = None)
rottenApple_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/RottenApples/RottenAppleBatch2Labeled/RottenApplesBatch2Labels.csv", header = None)
rottenApple_csv_batch_3 = pd.read_csv("/content/Fruit Defects Dataset /Train/RottenApples/RottenAppleBatch3Labaled/RottenApplesBatch3Labels.csv", header = None)
rottenStrawberry_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/RottenStrawberries/Batch1RottenStrawBerryLabels/RottenStrawberriesBatch1Labels.csv", header = None)
rottenStrawberry_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/RottenStrawberries/Batch2RottenStrawBerryLabels/RottenStrawBerryBatch2.csv", header = None)
rottenStrawberry_csv_batch_3 = pd.read_csv("/content/Fruit Defects Dataset /Train/RottenStrawberries/Batch3RottenStrawberrylabel/rottenStrawberryBtch3labels.csv", header = None)
freshApples_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/FreshApples/FreshApplebtch2label/FreshApplesBatch2LabelsFresh.csv", header = None)
freshApples_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/FreshApples/FreshApplesBatch1Labels/FreshAppleBatch1Labels.csv", header = None)
rottenTomato_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/Rotten Tomatoes/Rotten TomatoBatch1/Batch1TomoatosLabelsBbox.csv", header = None)
rottenTomato_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/Rotten Tomatoes/RottnTomatoBatch2/RottenTomatyoBatch2Labelss.csv", header = None)
rottenTomato_csv_batch_3 = pd.read_csv("/content/Fruit Defects Dataset /Train/Rotten Tomatoes/RottenTomatBtch3/RottenTomatoesBatch3Labssles.csv", header = None)
rottenTomato_csv_batch_4 = pd.read_csv("/content/Fruit Defects Dataset /Train/Rotten Tomatoes/RottenTomatoesBatch4/Tomatobatch4labelssRotten.csv", header = None)
freshTomato_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/Fresh Tomatoes/FreshTomatoesBatch1Labelss/FreshTomatoesLabelsBatch1Labels.csv", header = None)
freshTomato_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/Fresh Tomatoes/FreshTomatBatch2Labessls/Batch2TomatlabelsFresh.csv", header = None)
rottenPeach_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/Rotten Peaches/Bctch1RottenPeachesLabels/RottenPeachesBatch1Labelks.csv", header = None)
rottenPeach_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/Rotten Peaches/RottenPeachesBatch2lbeLabels/Batch2PeahcesRottenLabels.csv", header = None)
freshPeach_csv_batch_1 = pd.read_csv("/content/Fruit Defects Dataset /Train/Peaches/Batch1FreshPeaches/freshpeahcheslabelsbatch1.csv", header = None)
freshPeach_csv_batch_2 = pd.read_csv("/content/Fruit Defects Dataset /Train/Peaches/Batch2FreshPeache/Batch2FreshPeachesLabels.csv", header = None)

strawberry_csv_batch_3.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
strawberry_csv_batch_2.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
strawberry_csv_batch_1.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenApple_csv_batch_1.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenApple_csv_batch_2.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenApple_csv_batch_3.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenStrawberry_csv_batch_1.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenStrawberry_csv_batch_2.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenStrawberry_csv_batch_3.columns = ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
freshApples_csv_batch_2.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
freshApples_csv_batch_1.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenTomato_csv_batch_1.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenTomato_csv_batch_2.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenTomato_csv_batch_3.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
rottenTomato_csv_batch_4.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
#freshTomato_csv_batch_1.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
#freshTomato_csv_batch_2.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
#rottenPeach_csv_batch_1.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
#rottenPeach_csv_batch_2.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
#freshPeach_csv_batch_1.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]
#freshPeach_csv_batch_2.columns =  ["Fruit", "Coord1", "Coord2", "Coord3", "Coord4", "Image_id", "OneSize", "TwoSize"]

#Drop some watermark data for Fresh StrawBerry Batch 1 Labeled images [59, 9, 93]

# strawberry_csv_batch_1 = strawberry_csv_batch_1[Image_id not in ["FreshStrawberries59.jpeg, FreshStrawberries9.jpeg, FreshStrawberries93.jpeg"]]
strawberry_csv_batch_1.drop(strawberry_csv_batch_1[strawberry_csv_batch_1["Image_id"] == "FreshStrawberries59.jpeg"].index, inplace = True)
strawberry_csv_batch_1.drop(strawberry_csv_batch_1[strawberry_csv_batch_1["Image_id"] == "FreshStrawberries9.jpeg"].index, inplace = True)
strawberry_csv_batch_1.drop(strawberry_csv_batch_1[strawberry_csv_batch_1["Image_id"] == "FreshStrawberries93.jpeg"].index, inplace = True)
freshTomato_csv_batch_1.drop(freshTomato_csv_batch_1[freshTomato_csv_batch_1["Image_id"] == "Fresh Tomatoes66AddonPart1.jpeg"].index, inplace = True)

strawberry_csv_batch_1 = strawberry_csv_batch_1.reset_index(drop=True)
freshTomato_csv_batch_1 = freshTomato_csv_batch_1.reset_index(drop = True)

#Stack all the csv files together. 
list_of_all_dataframes = [strawberry_csv_batch_1, strawberry_csv_batch_2, strawberry_csv_batch_3, rottenApple_csv_batch_1, 
                          rottenApple_csv_batch_2, rottenApple_csv_batch_3, rottenStrawberry_csv_batch_1, rottenStrawberry_csv_batch_2, 
                          rottenStrawberry_csv_batch_3, freshApples_csv_batch_2, freshApples_csv_batch_1, rottenTomato_csv_batch_1, 
                          rottenTomato_csv_batch_2, rottenTomato_csv_batch_3, rottenTomato_csv_batch_4, freshTomato_csv_batch_1, 
                          freshTomato_csv_batch_2, rottenPeach_csv_batch_1, rottenPeach_csv_batch_2, freshPeach_csv_batch_1, freshPeach_csv_batch_2]
fruit_df = pd.concat(list_of_all_dataframes, ignore_index = True)

total_row_sum_check = 0 
for dataframe in list_of_all_dataframes:
  total_row_sum_check += dataframe.shape[0]
print("Checked total rows from all the dataframes combined: {}".format(total_row_sum_check))

def run_dataframe_check(after_mis = True):
  assert total_row_sum_check == fruit_df.shape[0]
  if after_mis:
    print("DataFrame shape: {}".format(fruit_df.shape))
    print("Unique Fruit Labels {}".format(fruit_df["Fruit"].unique()))
    print("Number of Unique Images {}".format(len(fruit_df["Image_id"].unique())))

run_dataframe_check(after_mis = False)

#Specify more image types when 
def more_specific_Image_id(image_id, fruit):
  if fruit == "Bad_Spots":
    if re.search("RottenStrawberries", image_id):
      return "Strawberry_Bad_Spot"
    elif re.search("RottenApples", image_id):
      return "Apple_Bad_Spot"
    elif re.search("Rotten Tomatoes", image_id):
      return "Tomato_Bad_Spot"
    elif re.search("Rotten Peaches", image_id):
      return "Peaches_Bad_Spot"
    else:
      raise ValueError("Could not find a match for some of the Image_ids")

  else:
    return fruit

def fix_fruitdf_mistakes(fruit):

  if fruit == "Peach ":
    return "Peach"
  else:
    return fruit

fruit_df["Fruit"] = fruit_df.apply(lambda row: more_specific_Image_id(row.Image_id, row.Fruit), axis = 1)
fruit_df["Fruit"] = fruit_df.apply(lambda row: fix_fruitdf_mistakes(row.Fruit), axis = 1)

run_dataframe_check()

#Post Processing 
fruit_df = fruit_df[fruit_df["Image_id"] != "FreshStrawberries15.jpeg"]

bounding_box_dict = dict()
labels_dict = dict()
classes = ["Apples", "Strawberry", "Apple_Bad_Spot", "Strawberry_Bad_Spot"]
print(classes)
# classes = ["Bad_Spots", "Strawberry", "Apples"]

for row_index in range(len(fruit_df)): 
  current_image_file = fruit_df.iloc[row_index]["Image_id"]
  if current_image_file not in bounding_box_dict:
    bounding_box_dict[current_image_file] = list()
    labels_dict[current_image_file] = list()
  bounding_box_dict[current_image_file].append(fruit_df.iloc[row_index, 1:5].to_list())
  labels_dict[current_image_file].append(classes.index(fruit_df.iloc[row_index, 0]))

print(len(bounding_box_dict))
print(len(labels_dict))



NameError: ignored

## Class function + util functions

In [ ]:
# COCO Bounding box: (x-top left, y-top left, width, height)
# Pascal VOC Bounding box :(x-top left, y-top left,x-bottom right, y-bottom right)

In [ ]:
def ffile_path(image_id, full_image_file_paths):
  for image_path in full_image_file_paths:
    if image_id in image_path:
      return image_path

def Coco_Create(labels_dict_fn,bounding_box_dict_fn,imgs_key_fn):
  Coco_dict={}
  images_full_list=[]
  annotations_full_list = []
  bbox_id=0

  for index in range(len(bounding_box_dict_fn)):

        sub_annotation_dict={}

        image_key= imgs_key_fn[index]
        img_path = ffile_path(image_key, full_image_file_paths) 
        img = Image.open(img_path).convert("RGB")

        width, height = img.size

        images_sub_dict ={}

        images_sub_dict['file_name'] = image_key
        images_sub_dict['id'] = index
        images_sub_dict['height'] = height
        images_sub_dict['width'] = width
        images_full_list.append(images_sub_dict)
        boxesxywh = bounding_box_dict_fn[image_key]

        ann_nest_dict={}
        labels = labels_dict_fn[image_key]

        for i,box_xywh in enumerate(boxesxywh):
          bbox_id+=1
          
          x1= box_xywh[0]
          w= box_xywh[2]

          y1= box_xywh[1]
          h= box_xywh[2]

          x2 = w + x1
          y2 = h + y1 

          boxes_xyxy = [x1,y1,x2,y2]
          area = (y2 - y1) * (x2 - x1)

          box_xywh = [float(box_xywh[0]),
                      float(box_xywh[1]),
                      float(box_xywh[2]),
                      float(box_xywh[3])]

          ann_nest_dict['bbox'] = box_xywh
          ann_nest_dict['id'] = bbox_id
          ann_nest_dict['image_id'] = index
          ann_nest_dict['area'] = int(area)
          ann_nest_dict['iscrowd'] = 0
          ann_nest_dict['category_id'] = labels[i]+1
          annotations_full_list.append(ann_nest_dict)

  Coco_dict['info'] = {
          "year": "",
          "version": "",
          "description": "",
          "contributor": "",
          "url": "",
          "date_created": ""
      }
  Coco_dict['licenses'] = []
  Coco_dict['annotations'] = annotations_full_list
  Coco_dict['images'] = images_full_list
  categories= [{ "id" : 1, 
              "name" : 'Apples', 
              "supercategory" : 'Fruit'},
              
              {"id" : 2, 
              "name" : 'Strawberry', 
              "supercategory" : 'Fruit'},

            {"id" : 3, 
              "name" : 'Tomato', 
              "supercategory" : 'Fruit'},
            {"id" : 4, 
              "name" : 'Apple_Bad_Spot', 
              "supercategory" : 'Fruit'},
            {"id" : 5, 
              "name" : 'Strawberry_Bad_Spot', 
              "supercategory" : 'Fruit'},
            {"id" : 6, 
              "name" : 'Tomato_Bad_Spot',
              "supercategory" : 'Fruit'}]
  Coco_dict['categories'] = categories
  return Coco_dict

# Visualize the Coco Dataset, just to make sure everything seems ok

In [ ]:
!pip install fiftyone

In [ ]:
### For fiftyone package to visualize 
# This is where all the images get dumped into one directory
# Meaning that the train and test images are not split into seperate 
## Below is how it wants the images
 #   <coco>/
 #   data/
 #       <filename0>.<ext>
 #       <filename1>.<ext>
 #       ...
 #   labels.json

full_image_file_paths = glob.glob("/content/Fruit Defects Dataset /Train/*/*/*.jpeg")
imgs_key= sorted(bounding_box_dict.keys())

# This is creating the coco json file, not just for the train set, but for all images
Coco_dict_fiftyone= Coco_Create(labels_dict,bounding_box_dict,imgs_key)

os.makedirs(
  os.path.dirname(os.path.abspath('/content/coco/labels.json')), exist_ok=True
)
json.dump(Coco_dict_fiftyone, open('/content/coco/labels.json', "w"), indent=4)

if not os.path.exists("/content/coco/data"):
    os.mkdir("/content/coco/data")
for imageName in full_image_file_paths:
    shutil.copy(imageName, "/content/coco/data")

In [ ]:
import fiftyone as fo

# A name for the dataset
name = "Fruit-Defect-Dataset"

# The directory containing the dataset to import
dataset_dir = "/content/coco/"

# The type of the dataset being imported
# Any subclass of `fiftyone.types.Dataset` is supported
dataset_type = fo.types.COCODetectionDataset  # for example

dataset = fo.Dataset.from_dir(dataset_dir, dataset_type, name=name)

In [ ]:
# View summary info about the dataset
print(dataset)

# Print the first few samples in the dataset
print(dataset.head())

In [ ]:
session = fo.launch_app(dataset)

# If the session says "no dataset selected", click the FiftyOne logo on the top right of the display that comes up below. 
# Thats seems to launch the dataset for viewing.

## Train dataset

In [ ]:
### This is the format for the rwightman/efficientdet-pytorch package
 #   <cococ>/
 #   train2017/
 #       <filename0>.<ext>
 #       <filename1>.<ext>
 #       ...
 #   val2017/
 #       <filename0>.<ext>
 #       <filename1>.<ext>
 #       ...
 #   annotations/
 #       instances_train2017.json
 #       instances_val2017.json

# Running the Coco_Create function for train and test set
imgs_key= sorted(bounding_box_dict.keys())
imgs_key_train = imgs_key[:int(len(imgs_key) * 0.8)]
imgs_key_test = imgs_key[int(len(imgs_key) * 0.8):]

labels_dict_train = {k: labels_dict[k] for k in imgs_key_train}
bounding_box_dict_train = {k: bounding_box_dict[k] for k in imgs_key_train}
Coco_dict_train= Coco_Create(labels_dict_train,bounding_box_dict_train,imgs_key_train)

labels_dict_test = {k: labels_dict[k] for k in imgs_key_test}
bounding_box_dict_test = {k: bounding_box_dict[k] for k in imgs_key_test}
Coco_dict_test= Coco_Create(labels_dict_test,bounding_box_dict_test,imgs_key_test)

save_json_path_train ='/content/coco/annotations/instances_train2017.json'
save_json_path_val ='/content/coco/annotations/instances_val2017.json'

os.makedirs(
  os.path.dirname(os.path.abspath(save_json_path_train)), exist_ok=True
)
json.dump(Coco_dict_train, open(save_json_path_train, "w"), indent=4)

# For the time being I made the test set, the train set -- for code testing purposes
# In the future this should be changed to include a blinded dataset to the model
os.makedirs(
  os.path.dirname(os.path.abspath(save_json_path_val)), exist_ok=True
)
json.dump(Coco_dict_test, open(save_json_path_val, "w"), indent=4)

train_image_paths =[]
for path in full_image_file_paths:
  if os.path.basename(path) in imgs_key_train:
      train_image_paths.append(path)

test_image_paths =[]
for path in full_image_file_paths:
  if os.path.basename(path) in imgs_key_test:
      test_image_paths.append(path)

if not os.path.exists("/content/coco/train2017"):
    os.mkdir("/content/coco/train2017")
for imageName in train_image_paths:
    shutil.copy(imageName, "/content/coco/train2017")

if not os.path.exists("/content/coco/val2017"):
    os.mkdir("/content/coco/val2017")
for imageName in test_image_paths:
    shutil.copy(imageName, "/content/coco/val2017")

In [ ]:
# %cd  '/content/'

# !unzip '/content/efficientdet-pytorch-master.zip'
!git clone https://github.com/rwightman/efficientdet-pytorch.git
%cd  '/content/efficientdet-pytorch'

Cloning into 'efficientdet-pytorch'...
remote: Enumerating objects: 917, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 917 (delta 42), reused 35 (delta 15), pack-reused 839
Receiving objects: 100% (917/917), 362.07 KiB | 1.68 MiB/s, done.
Resolving deltas: 100% (585/585), done.
/content/efficientdet-pytorch


In [ ]:
!rm -rf "/content/coco"

In [ ]:
#!git clone https://github.com/rwightman/efficientdet-pytorch.git
%cd  '/content/efficientdet-pytorch'

!python train.py  /content/coco --dataset coco2017 --model efficientdet_d0 -b 32 --amp --lr 0.0005 --warmup-epochs 3 --opt momentum --num-classes 8 --model-ema --model-ema-decay 0.0003  --pretrained --epochs 80



/content/efficientdet-pytorch
Training with a single process on 1 GPU.
Model efficientdet_d0 created, param count: 3829793
Using native Torch AMP. Training in mixed precision.
Scheduled epochs: 90
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please

In [ ]:
import argparse
import time
import torch
import torch.nn.parallel
from contextlib import suppress

from effdet import create_model, create_evaluator, create_dataset, create_loader
from effdet.data import resolve_input_config
from timm.utils import AverageMeter, setup_default_logging
from timm.models.layers import set_layer_config

has_apex = False
try:
    from apex import amp
    has_apex = True
except ImportError:
    pass

has_native_amp = False
try:
    if getattr(torch.cuda.amp, 'autocast') is not None:
        has_native_amp = True
except AttributeError:
    pass

torch.backends.cudnn.benchmark = True


def validate(args):

    args['pretrained'] = args['pretrained'] or not args['checkpoint']  # might as well try to validate something
    args['prefetcher'] = not args['no_prefetcher']

    # create model
    with set_layer_config(scriptable=args['torchscript']):
        extra_args = {}
        if args['img_size'] is not None:
            extra_args = dict(image_size=(args['img_size'] ,args['img_size']))
        bench = create_model(
            args['model'],
            bench_task='predict',
            num_classes=args['num_classes'],
            pretrained=args['pretrained'],
            redundant_bias=args['redundant_bias'],
            soft_nms=args['soft_nms'],
            checkpoint_path=args['checkpoint'],
            checkpoint_ema=args['use_ema'],
            **extra_args,
        )
    model_config = bench.config

    param_count = sum([m.numel() for m in bench.parameters()])
    print('Model %s created, param count: %d' % (args['model'], param_count))

    bench = bench.cuda()

    amp_autocast = suppress
   

    dataset = create_dataset(args['dataset'], args['root'], args['split'])
    input_config = resolve_input_config(args, model_config)
    loader = create_loader(
        dataset,
        input_size=input_config['input_size'],
        batch_size=args['batch_size'],
        use_prefetcher=args['prefetcher'],
        interpolation=input_config['interpolation'],
        fill_color=input_config['fill_color'],
        mean=input_config['mean'],
        std=input_config['std'],
        num_workers=args['workers'],
        pin_mem=args['pin_mem'])

    evaluator = create_evaluator(args['dataset'], dataset, pred_yxyx=False)
    bench.eval()
    batch_time = AverageMeter()
    end = time.time()
    last_idx = len(loader) - 1
    with torch.no_grad():
        for i, (input, target) in enumerate(loader):

            with amp_autocast():
                #### HERE IS WHERE SAVING THE OUTPUTS CAN TAKE PLACE
                output = bench(input, img_info=target)
            evaluator.add_predictions(output, target)

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args['log_freq'] == 0 or i == last_idx:
                print(
                    'Test: [{0:>4d}/{1}]  '
                    'Time: {batch_time.val:.3f}s ({batch_time.avg:.3f}s, {rate_avg:>7.2f}/s)  '
                    .format(
                        i, len(loader), batch_time=batch_time,
                        rate_avg=input.size(0) / batch_time.avg)
                )

    mean_ap = 0.
    if dataset.parser.has_labels:
        mean_ap = evaluator.evaluate()
    else:
        evaluator.save(args.results)

    return mean_ap

args={}

args['dataset'] = 'coco2017'
args['num_classes'] = 6
args['pretrained'] = True
args['checkpoint'] = '/content/efficientdet-pytorch/output/train/20210615-213943-tf_efficientdet_d0/model_best.pth.tar'
args['redundant_bias'] = None
args['model'] = 'tf_efficientdet_d0'
args['workers'] = 4
args['batch_size'] = 128
args['soft_nms'] = None
args['use_ema'] = True
args['img_size'] = None
args['mean'] = None
args['std'] = None
args['interpolation'] = 'bilinear'
args['fill_color'] = None
args['no_prefetcher'] = False
args['pin_mem'] = False
args['torchscript'] = True
args['split'] ='val'
args['root'] = '/content/coco/'
args['log_freq'] = 10

validate(args)


Model tf_efficientdet_d0 created, param count: 3830927
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test: [   0/1]  Time: 2.562s (2.562s,   41.77/s)  
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.16s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.037
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.060
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.072
 Average Recall  

0.0001205984967999586